In [21]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier
import warnings

## Medical Cost Expenses

In [24]:
ins = pd.read_csv('insurance.csv')

In [26]:
ins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [28]:
X = ins.drop('charges',axis=1)
y = ins['charges']

In [ ]:
list(X.columns[X.dtypes==object])

In [39]:
cgbm = CatBoostRegressor(random_state=24, cat_features=['sex','smoker','region'] #can give like this -> list(X.columns[X.dtypes==object]))
params = {
         'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
         'learning_rate': np.linspace(0.001,1,6)         
}
          
          
         

kfold = KFold(n_splits = 5,shuffle = True,random_state =24)

In [41]:
gcv = GridSearchCV(cgbm,param_grid = params, cv = kfold, scoring = 'r2',verbose = 3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
0:	learn: 9063.6863526	total: 151ms	remaining: 7.39s
1:	learn: 6918.3022646	total: 159ms	remaining: 3.82s
2:	learn: 6097.7537587	total: 166ms	remaining: 2.61s
3:	learn: 5462.7615179	total: 180ms	remaining: 2.07s
4:	learn: 5126.3778156	total: 193ms	remaining: 1.74s
5:	learn: 5004.5121744	total: 206ms	remaining: 1.51s
6:	learn: 4805.8254760	total: 219ms	remaining: 1.35s
7:	learn: 4738.0701801	total: 232ms	remaining: 1.22s
8:	learn: 4637.0544066	total: 245ms	remaining: 1.12s
9:	learn: 4579.7119064	total: 258ms	remaining: 1.03s
10:	learn: 4569.0567212	total: 271ms	remaining: 960ms
11:	learn: 4522.2595482	total: 283ms	remaining: 898ms
12:	learn: 4507.6051192	total: 296ms	remaining: 843ms
13:	learn: 4496.9503713	total: 309ms	remaining: 795ms
14:	learn: 4491.4697798	total: 322ms	remaining: 751ms
15:	learn: 4489.9433578	total: 335ms	remaining: 711ms
16:	learn: 4477.6441900	total: 347ms	remaining: 674ms
17:	learn: 4475.3176953	total

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001EF4F23F7D0>,
             n_jobs=-1,
             param_grid={'learning_rate': array([0.001 , 0.2008, 0.4006, 0.6004, 0.8002, 1.    ]),
                         'max_depth': [2, 3, 4],
                         'n_estimators': [10, 20, 30, 50, 100, 200]},
             scoring='r2', verbose=3)

In [43]:
print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 0.4006, 'max_depth': 2, 'n_estimators': 50}
0.8602345444785048


In [49]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [47]:
model = gcv.best_estimator_

In [51]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)


0:	learn: 8770.4088591	total: 10.7ms	remaining: 522ms
1:	learn: 7029.0718773	total: 25.3ms	remaining: 606ms
2:	learn: 6171.6811232	total: 33.8ms	remaining: 530ms
3:	learn: 5506.2580675	total: 48.8ms	remaining: 561ms
4:	learn: 5317.9826191	total: 62.9ms	remaining: 566ms
5:	learn: 5126.7841442	total: 76.7ms	remaining: 563ms
6:	learn: 5071.4322952	total: 84ms	remaining: 516ms
7:	learn: 4998.1561701	total: 97.7ms	remaining: 513ms
8:	learn: 4846.9575511	total: 111ms	remaining: 506ms
9:	learn: 4814.1874831	total: 124ms	remaining: 498ms
10:	learn: 4774.5970371	total: 138ms	remaining: 490ms
11:	learn: 4657.1137693	total: 152ms	remaining: 482ms
12:	learn: 4638.8465974	total: 166ms	remaining: 472ms
13:	learn: 4592.3906596	total: 180ms	remaining: 462ms
14:	learn: 4567.6154609	total: 195ms	remaining: 454ms
15:	learn: 4556.5706760	total: 209ms	remaining: 445ms
16:	learn: 4549.0216723	total: 229ms	remaining: 445ms
17:	learn: 4544.5264453	total: 244ms	remaining: 433ms
18:	learn: 4536.2238316	total: 2

0.8713059252223407

In [53]:
r2_score(y_test,y_pred)

0.8713059252223407